In [ ]:
# -*- coding: utf-8 -*-
"""
Real-Time Lane Detection with Interactive UI in Google Colab

This script provides a basic implementation of real-time lane detection
using OpenCV, now enhanced with an interactive user interface built with ipywidgets.
It processes video frames, applies image processing techniques to identify
lane lines, and overlays them back onto the original frames.

Key steps involved:
1.  **Grayscale Conversion:** Convert the input frame to grayscale for simpler processing.
2.  **Gaussian Blur:** Apply a Gaussian blur to reduce noise and smooth the image.
3.  **Canny Edge Detection:** Detect edges in the image, which are crucial for identifying lane boundaries.
4.  **Region of Interest (ROI) Masking:** Define a polygonal region to focus on the road area,
    ignoring irrelevant parts of the image.
5.  **Hough Transform:** Use the Probabilistic Hough Line Transform to detect straight lines
    within the ROI, which are potential lane lines.
6.  **Lane Line Averaging:** Process the detected lines to separate them into left and right
    lane lines, calculate their average slope and intercept, and extrapolate them to
    form continuous lines.
7.  **Overlay:** Draw the detected lane lines onto the original video frame.
8.  **Interactive UI:** Uses `ipywidgets` for file upload and a processing button.
9.  **Improved Video Playback & Download:** Uses a compatible codec and provides a direct download link.
"""

import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML, display, clear_output, FileLink
from base64 import b64encode
import os
import ipywidgets as widgets
from ipywidgets import FileUpload, Button, Output, VBox

# Function to display images in Colab (as cv2.imshow doesn't work directly)
def cv2_imshow(image):
    """
    Displays an OpenCV image using Matplotlib, suitable for Google Colab.
    """
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

def region_of_interest(img, vertices):
    """
    Applies an image mask.

    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    mask = np.zeros_like(img)
    # Filling pixels inside the polygon defined by "vertices" with the fill color
    cv2.fillPoly(mask, vertices, 255)
    # Returning the image only where mask pixels are non-zero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines, color=(0, 255, 0), thickness=5):
    """
    Draws lines on an image.
    """
    if lines is None:
        return img
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(line_img, (x1, y1), (x2, y2), color, thickness)
    return cv2.addWeighted(img, 0.8, line_img, 1.0, 0.0)

def make_coordinates(image, line_parameters):
    """
    Calculates the coordinates for a line given its slope and intercept.
    """
    if line_parameters is None:
        return None
    slope, intercept = line_parameters
    y1 = image.shape[0]  # Bottom of the image
    y2 = int(y1 * 3 / 5)  # Approximately 3/5th from the bottom
    x1 = int((y1 - intercept) / slope)
    x2 = int((y2 - intercept) / slope)
    return np.array([x1, y1, x2, y2])

def average_slope_intercept(image, lines):
    """
    Averages the slopes and intercepts of the detected lines to find
    the best fit for the left and right lane lines.
    """
    left_fit = []
    right_fit = []
    if lines is None:
        return None, None

    for line in lines:
        x1, y1, x2, y2 = line.reshape(4)
        parameters = np.polyfit((x1, x2), (y1, y2), 1)
        slope = parameters[0]
        intercept = parameters[1]
        if slope < 0:  # Negative slope for left lane
            left_fit.append((slope, intercept))
        else:  # Positive slope for right lane
            right_fit.append((slope, intercept))

    left_fit_average = np.average(left_fit, axis=0) if left_fit else None
    right_fit_average = np.average(right_fit, axis=0) if right_fit else None

    left_line = make_coordinates(image, left_fit_average)
    right_line = make_coordinates(image, right_fit_average)

    return left_line, right_line

def process_frame(image):
    """
    Processes a single frame to detect and draw lane lines.
    """
    # 1. Grayscale conversion
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 2. Gaussian Blur
    blur_image = cv2.GaussianBlur(gray_image, (5, 5), 0)

    # 3. Canny Edge Detection
    canny_image = cv2.Canny(blur_image, 50, 150) # Low and high thresholds

    # 4. Region of Interest Masking
    height = image.shape[0]
    width = image.shape[1]
    # Define a trapezoidal region of interest
    vertices = [
        (int(width * 0.1), height),
        (int(width * 0.45), int(height * 0.6)),
        (int(width * 0.55), int(height * 0.6)),
        (int(width * 0.9), height)
    ]
    cropped_image = region_of_interest(canny_image, np.array([vertices], np.int32))

    # 5. Hough Transform
    # min_line_length: minimum number of pixels making up a line
    # max_line_gap: maximum gap in pixels between connectable line segments
    lines = cv2.HoughLinesP(cropped_image, 2, np.pi/180, 100, np.array([]), minLineLength=40, maxLineGap=5)

    # 6. Lane Line Averaging
    left_line, right_line = average_slope_intercept(image, lines)
    averaged_lines = []
    if left_line is not None:
        averaged_lines.append(left_line)
    if right_line is not None:
        averaged_lines.append(right_line)

    # Reshape for draw_lines function
    if averaged_lines:
        lines_to_draw = np.array(averaged_lines).reshape(-1, 1, 4)
    else:
        lines_to_draw = None

    # 7. Overlay
    line_image = draw_lines(image, lines_to_draw)
    return line_image

def display_video_in_colab(video_path):
    """
    Displays a local video file in Google Colab using HTML5 video tag.
    """
    if not os.path.exists(video_path):
        print(f"Error: Output video '{video_path}' not found for display.")
        return

    # Check if the file is empty
    if os.path.getsize(video_path) == 0:
        print(f"Error: Output video '{video_path}' is empty. No frames were written.")
        return

    mp4 = open(video_path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    HTML_output = f"""
    <video width="640" height="480" controls>
        <source src="{data_url}" type="video/mp4">
        Your browser does not support the video tag.
    </video>
    """
    display(HTML(HTML_output))

# --- Interactive UI Functions ---

# Output widget to display messages and results
output_area = Output()

def on_process_button_clicked(b):
    """
    Handles the click event of the process button.
    Reads the uploaded file, processes it, and displays the result.
    """
    with output_area:
        clear_output() # Clear previous output
        print("Starting video processing...")
        if not file_upload.value:
            print("Please upload a video file first.")
            return

        # Get the uploaded file data
        uploaded_file_name = list(file_upload.value.keys())[0]
        uploaded_file_content = file_upload.value[uploaded_file_name]['content']

        # Save the uploaded file temporarily
        input_video_path = uploaded_file_name
        try:
            with open(input_video_path, 'wb') as f:
                f.write(uploaded_file_content)
            print(f"Uploaded video saved as: {input_video_path}")
        except Exception as e:
            print(f"Error saving uploaded file: {e}")
            return

        output_video_name = 'lane_detection_output.mp4'

        print(f"Processing video: {input_video_path}...")

        cap = cv2.VideoCapture(input_video_path)

        if not cap.isOpened():
            print(f"Error: Could not open video file {input_video_path}. Please ensure it's a valid video format.")
            # Clean up the temporary input file if it was created
            if os.path.exists(input_video_path):
                os.remove(input_video_path)
            return

        # Get video properties
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = int(cap.get(cv2.CAP_PROP_FPS))

        # Define the codec and create VideoWriter object
        # Changed codec to 'mp4v' (MPEG-4) which is often more widely supported
        fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Previously 'avc1'
        out = cv2.VideoWriter(output_video_name, fourcc, fps, (frame_width, frame_height))

        if not out.isOpened():
            print(f"Error: Could not create video writer for {output_video_name}. Codec '{fourcc}' might not be supported.")
            print("You might try a different video format for your input or another codec like 'XVID'.")
            # Clean up the temporary input file
            if os.path.exists(input_video_path):
                os.remove(input_video_path)
            return

        frame_count = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            processed_frame = process_frame(frame)
            out.write(processed_frame)

            frame_count += 1
            if frame_count % 100 == 0:
                print(f"Processed {frame_count} frames...")

        cap.release()
        out.release()
        print(f"Video processing complete. Output saved to {output_video_name}")

        # Display the processed video (attempting again, but download is primary)
        display_video_in_colab(output_video_name)

        # Provide a direct download link for the processed video
        if os.path.exists(output_video_name) and os.path.getsize(output_video_name) > 0:
            print("\n--- Processing Complete! ---")
            print("You should see the video player above (if compatible) and a download link below.")
            print("If the video player is black or not visible, please use the download link.")
            print("\n➡️ **Download your processed video here:**")
            display(FileLink(output_video_name)) # This should display the link
            print("\n(The video file will remain available for download until this Colab session ends or you delete it manually.)")
        else:
            print("\n--- Processing Complete, but output video is missing or empty. ---")
            print("This usually indicates an an issue during video writing. Please check the console for errors.")


        # Clean up the temporary files after display attempt
        if os.path.exists(input_video_path):
            os.remove(input_video_path)
        # Keep the output video for download until the session ends or user deletes it
        # if os.path.exists(output_video_name):
        #     os.remove(output_video_name)
        print("Temporary input file cleaned up. Output video remains for download.")


# Create widgets
file_upload = FileUpload(
    accept='.mp4,.avi,.mov', # Accepted file types
    multiple=False,         # Allow only one file upload
    description='Upload Video'
)

process_button = Button(description="Process Video")
process_button.on_click(on_process_button_clicked)

# Arrange widgets in a vertical box
ui = VBox([file_upload, process_button, output_area])

# Display the UI
if __name__ == "__main__":
    print("Please upload a video file using the widget below and click 'Process Video'.")
    display(ui)

Please upload a video file using the widget below and click 'Process Video'.
